In [1]:
import glob
import pandas as pd
import time
import docx
import PyPDF2
import cv2
import pytesseract
from datetime import date
import re
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

In [2]:
DF = pd.DataFrame(columns=["fileName", "text"])
st_time = time.time()
paths = glob.glob(r"C:\Users\QuadAdmin\Downloads\files\**\*.jpg",recursive = True)
print(f"time took: {(time.time() - st_time):.2f} seconds")

time took: 0.00 seconds


In [3]:
paths

['C:\\Users\\QuadAdmin\\Downloads\\files\\images\\call11.jpg',
 'C:\\Users\\QuadAdmin\\Downloads\\files\\images\\call7.jpg',
 'C:\\Users\\QuadAdmin\\Downloads\\files\\images\\f018.jpg',
 'C:\\Users\\QuadAdmin\\Downloads\\files\\images\\f019.jpg']

In [4]:
DF["fileName"] = paths

In [5]:
DF

,fileName,text
0,C:\Users\QuadAdmin\Downloads\files\images\call...,NaN
1,C:\Users\QuadAdmin\Downloads\files\images\call...,NaN
2,C:\Users\QuadAdmin\Downloads\files\images\f018...,NaN
3,C:\Users\QuadAdmin\Downloads\files\images\f019...,NaN


In [6]:
import findspark
findspark.init()

In [7]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [8]:
conf = SparkConf()
conf.set("spark.network.timeout", "1000000")
conf.set("spark.executor.heartbeatInterval", "1000000")
conf.set("spark.speculation","false")
conf.set("spark.app.name","blaher")
spark = SparkSession.builder.config(conf = conf).master("local[1]").enableHiveSupport().getOrCreate() #("local[*]") uses all the cpu cores#

In [9]:
spark

In [10]:
sc = spark.sparkContext

In [11]:
# ls_ran = list(range(1,41))
# len(ls_ran)

In [12]:
# ls_rdd = sc.parallelize(ls_ran)
# ls_rdd.take(5)

In [13]:
# type(ls_rdd)

In [14]:
# ls_rdd.getNumPartitions()

In [15]:
def getText_jpg(filename):
    img = cv2.imread(filename)
    text = pytesseract.image_to_string(img)
    # '\n'.join(fullText)
    return text

In [16]:
DF["fileName"][0]

'C:\\Users\\QuadAdmin\\Downloads\\files\\images\\call11.jpg'

In [53]:
# readfiles(DF["fileName"][0])
start_time = time.time()
re.sub("\n"," ",getText_jpg(DF["fileName"][3]))
print(f"time took: {(time.time() - start_time):.2f} seconds")
# getText_jpg(DF["fileName"][3])

time took: 0.82 seconds


In [18]:
paths_rdd = sc.parallelize(paths)

In [19]:
# text_rdd = paths_rdd.map(readfiles)

In [20]:
start_time = time.time()
DF["text"] = DF["fileName"].apply(lambda path: getText_jpg(path))
print(f"time took: {(time.time() - start_time):.2f} seconds")

time took: 3.02 seconds


In [21]:
DF

,fileName,text
0,C:\Users\QuadAdmin\Downloads\files\images\call...,"Customer: Hi there, | bought this computer in ..."
1,C:\Users\QuadAdmin\Downloads\files\images\call...,GR:\nCUNT:\n\nsR:\nCUNT:\nsR:\nCUNT:\nGSR:\n\n...
2,C:\Users\QuadAdmin\Downloads\files\images\f018...,Raghuram Rajan is the Katherine Dusak Miller D...
3,C:\Users\QuadAdmin\Downloads\files\images\f019...,Natural language processing is a massive field...


In [22]:
DF["text"] = DF["text"].apply(lambda text: re.sub("\n"," ",text))

In [23]:
DF

,fileName,text
0,C:\Users\QuadAdmin\Downloads\files\images\call...,"Customer: Hi there, | bought this computer in ..."
1,C:\Users\QuadAdmin\Downloads\files\images\call...,GR: CUNT: sR: CUNT: sR: CUNT: GSR: NT: SR: ...
2,C:\Users\QuadAdmin\Downloads\files\images\f018...,Raghuram Rajan is the Katherine Dusak Miller D...
3,C:\Users\QuadAdmin\Downloads\files\images\f019...,Natural language processing is a massive field...


In [24]:
from pyspark.sql.types import *

In [25]:
from pyspark.sql import *

In [26]:
import pyspark.sql.functions as F

In [27]:
from parse_pii.piiparser import *

In [28]:
DF["text"][0]

"Customer: Hi there, | bought this computer in your shop, but it doesn’t perform as well as described on the website. I chose my platform and the games | would be playing, but the computer doesn’t come anywhere close to hitting the frame rate that the manufacturer said it would even on the lowest graphic settings. I'd like to return it.  Representative: Hey there! No problem. You can return it or exchange it for a different computer. Many of our customers are serious gamers, and they speak highly of this (other) model. Why don’t you demo it and tell us what you think?  Customer: Wow! This computer's great! | hope | won't regret it if | do an exchange. \x0c"

In [29]:
classify(DF["text"][0])

'no'

In [30]:
sparkDF = spark.createDataFrame(DF)

In [31]:
sparkDF.show(5)

+--------------------+--------------------+
|            fileName|                text|
+--------------------+--------------------+
|C:\Users\QuadAdmi...|Customer: Hi ther...|
|C:\Users\QuadAdmi...|GR: CUNT:  sR: CU...|
|C:\Users\QuadAdmi...|Raghuram Rajan is...|
|C:\Users\QuadAdmi...|Natural language ...|
+--------------------+--------------------+



In [32]:
sparkDF.rdd.getNumPartitions()

1

In [33]:
classify_udf = F.udf(classify, StringType())

In [34]:
# start_time = time.time()
# sparkDF.select("fileName", classify_udf("text").alias("has_pii")).show(5)
# print(f"time took: {(time.time() - start_time):.2f} seconds")

In [35]:
# fDF = sparkDF.select("fileName", classify_udf("text").alias("has_pii"))

In [36]:
# start_time = time.time()
# fDF.coalesce(2).write.mode("overwrite").format("csv").option("header","true").save("output")
# print(f"time took: {(time.time() - start_time):.2f} seconds")

In [37]:
# fDF.printSchema()

In [38]:
import spacy
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import SpacyNlpEngine
from presidio_analyzer.recognizer_registry import RecognizerRegistry

In [39]:
def entity_counter(entity_list):
    freq = {}
    entity_list = sorted(entity_list)
    for items in entity_list: 
        freq[items] = entity_list.count(items) 
 
    counter = ""
    for key, value in freq.items(): 
        if key == entity_list[-1]:
            counter += f"{key}:{value}"
        else:
            counter += f"{key}:{value}"+", "
    return counter

In [40]:
analyzer = AnalyzerEngine(nlp_engine=nlp)

In [41]:
def pii_classify(stext):
    text = stext
    response = analyzer.analyze(correlation_id=0, text = text,language="en",entities=[], score_threshold=0.5)
    if len(response) == 0:
        return "no"
    else:
        return "yes"

In [42]:
(DF["text"][0])

"Customer: Hi there, | bought this computer in your shop, but it doesn’t perform as well as described on the website. I chose my platform and the games | would be playing, but the computer doesn’t come anywhere close to hitting the frame rate that the manufacturer said it would even on the lowest graphic settings. I'd like to return it.  Representative: Hey there! No problem. You can return it or exchange it for a different computer. Many of our customers are serious gamers, and they speak highly of this (other) model. Why don’t you demo it and tell us what you think?  Customer: Wow! This computer's great! | hope | won't regret it if | do an exchange. \x0c"

In [43]:
pii_classify(DF["text"][0])

'no'

In [44]:
pii_apply_udf = F.udf(pii_classify, StringType())

In [45]:
sparkDF.select("fileName", classify_udf("text").alias("has_pii")).show(4)

+--------------------+-------+
|            fileName|has_pii|
+--------------------+-------+
|C:\Users\QuadAdmi...|     no|
|C:\Users\QuadAdmi...|    yes|
|C:\Users\QuadAdmi...|    yes|
|C:\Users\QuadAdmi...|     no|
+--------------------+-------+



In [46]:
fDF = sparkDF.select("fileName", classify_udf("text").alias("has_pii"))

In [47]:
fDF.write.mode("overwrite").saveAsTable("op_table3")

In [48]:
# fDF.repartition(1).write.mode("overwrite").csv("output",header=True)

In [49]:
# start_time = time.time()
# sparkDF.coalesce(2).write.mode("overwrite").format("csv").option("header","true").save("output")
# print(f"time took: {(time.time() - start_time):.2f} seconds")